In [ ]:
# LATEST AND LAST UPDATES HAVING THE THIRD TAB
###########UPDATE ON 10TH NOVEMBER ################
# to incorporate the terminol0gies also 
####latest edited codes with a new front end #####################
import tkinter as tk
import numpy as np
from tkinter import filedialog, messagebox, Checkbutton, IntVar
from tkinter import ttk
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import re
import time 
import sys
from pathlib import Path
import os
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import os
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from transformers import BartForSequenceClassification, BartTokenizer

def resource_path(relative_path):
    """Get absolute path to resource, works for dev and for PyInstaller exe"""
    try:
        if getattr(sys, 'frozen', False):  # If running as .exe
            base_path = Path(sys.executable).parent
        else:  # If running in a development environment
            base_path = Path(__file__).parent
    except Exception as e:
        base_path = Path(os.path.abspath(os.path.dirname(__file__)))

    return base_path / relative_path

    
def run_selenium_automation(study_name,programmer_name):

    timeout = 10
    end_time = time.time() + timeout
    # Initialize the WebDriver
    driver = webdriver.Chrome()
    # options = Options()
    # options.add_experimental_option("detach", True) 
    # options.add_argument("--headless")
    # options.add_argument('--no-sandbox')
    # options.add_argument('--disable-gpu')
    # driver = webdriver.Chrome(options=options)
    
    # Replace 'your_pinnacle_21_link' with the actual URL
    driver.get('https://pfizer.pinnacle21.net/login?returnPath=%2F')
    
    try:
        # Wait for the "Log in with Single Sign-On" button to be clickable and click it
        login_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, 'sso_login'))
        )
        login_button.click()
        # time.sleep(10)
        # Wait for the table with studies to be visible
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="your-data-packages"]'))
        )
        # time.sleep(10)
        # Click on the link with the text "C5241016_SDTM"
        WebDriverWait(driver, 40).until(
            EC.element_to_be_clickable((By.LINK_TEXT, study_name))
        ).click()
        time.sleep(10)
        # Wait for the "View Issues" button to be clickable and click it
        view_issues_button = WebDriverWait(driver, 60).until(
            EC.element_to_be_clickable((By.XPATH, '//a[@class="btn btn-link widget-title-link" and text()="View Issues"]'))
        )
        view_issues_button.click()
        time.sleep(10)
            # Wait for the first issue to be present and click on it
        first_issue = WebDriverWait(driver, 100).until(
            EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div[2]/div[2]/div[2]/div[2]/div[1]/div[1]/div/div[2]/div[1]/div[2]/table/tbody/tr[2]/td[3]'))
        )
        first_issue.click()
        # Load the Excel file with the corpus
        corpus_path = resource_path("CORPUS_P219.xlsx")
        corpus_df = pd.read_excel(corpus_path)
        # corpus_df = pd.read_excel(r'C:\Users\SRIVAA86\OneDrive - Pfizer\Desktop\P21_AUTO\CORPUS_P219.xlsx')  # Replace with the actual path to your Excel file
        time.sleep(10)
        while True:
            try:
                # Extract the title
                title = WebDriverWait(driver, 60).until(
                    EC.presence_of_element_located((By.XPATH, '/html/body/div[5]/div/div/div/div[1]/div[1]/div/div[1]/div/section[1]/h2'))
                ).text.strip()
                time.sleep(10)
                # Navigate to the area with the description
                description_area = WebDriverWait(driver, 40).until(
                    EC.presence_of_element_located((By.XPATH, '/html/body/div[5]/div/div/div/div[1]/div[1]/div/div[1]/div/section[1]/aside[1]'))
                )
                # Extract the description
                description = description_area.find_element(By.XPATH, '/html/body/div[5]/div/div/div/div[1]/div[1]/div/div[1]/div/section[1]/aside[1]/div[1]').text
                time.sleep(10)
                dataset_area = WebDriverWait(driver, 60).until(
                    EC.presence_of_element_located((By.XPATH, '/html/body/div[5]/div/div/div/div[1]/div[1]/div/div[1]/div/section[1]/aside[2]'))
                )
                # Extract the dataset name
                dataset = dataset_area.find_element(By.XPATH, '/html/body/div[5]/div/div/div/div[1]/div[1]/div/div[1]/div/section[1]/aside[2]/div[1]')
                dataset_text = dataset.text.strip()  # Strip leading and trailing spaces
    
                # Remove the word "Dataset" if it exists in the text
                if "Dataset" in dataset_text:
                    dataset_text = dataset_text.replace("Dataset", "").strip()  # Replace and strip again
                time.sleep(10)
                # Navigate to the bottom panel area
                bottom_panel = WebDriverWait(driver, 80).until(
                    EC.presence_of_element_located((By.XPATH, '/html/body/div[5]/div/div/div/div[1]/div[3]'))
                )
                # Extract the table header
                table_header = bottom_panel.find_element(By.XPATH, '/html/body/div[5]/div/div/div/div[1]/div[3]/div/div[2]').text
                time.sleep(10)
                # Filter out numbers and specific words from the table header
                # filtered_table_header = ' '.join([word for word in table_header.split() 
                #                                   if not re.match(r'\d+', word) and word.lower() not in ['variable','dataset','cmdosu','total','cmroute','order', 'length','tsparmcd','lbtestcd','failures','%','affected','records','lbtest','lborresu','egstresu','epoch','ecdosu','egorresu','egtest','egtestcd','exdosu','lbstresu','<','pcspccnd','vsorresu','vsstresu','vstest','vstestcd','qscat']])
    
                #####################################################################
                if 'TREATMENT' in table_header.upper():
                # Chunk 1: Logic for handling TREATMENT with single-digit numbers
                    exclude_keywords = [
                        'variable', 'dataset', 'cmdosu', 'total', 'cmroute', 'order', 'length', 'tsparmcd', 'lbtestcd',
                        'failures', '%', 'affected', 'records', 'lbtest', 'lborresu', 'egstresu', 'epoch', 'ecdosu',
                        'egorresu', 'egtest', 'egtestcd', 'exdosu', 'lbstresu', '<', 'pcspccnd', 'vsorresu', 'vsstresu',
                        'vstest', 'vstestcd', 'qscat','aeout'
                    ]
                    keywords_allowing_digits = ['treatment']
                    
                    filtered_table_header = []
                    words = table_header.split()
                    
                    i = 0
                    while i < len(words):
                        word = words[i].lower()
                        if word in exclude_keywords:
                            i += 1
                            continue
                        if word in keywords_allowing_digits and i + 1 < len(words):
                            next_word = words[i + 1]
                            if re.match(r'^\d$', next_word):  # Check if the next word is a single digit
                                filtered_table_header.append(f"{words[i].upper()} {next_word}")
                                i += 1  # Skip the next word since it's already added
                        i += 1
                    
                    filtered_table_header = ' '.join(filtered_table_header)
                else:
                # Chunk 2: Generic filtering logic
                    filtered_table_header = ' '.join([
                        word for word in table_header.split()
                        if not re.match(r'\d+', word) and word.lower() not in [
                            'variable', 'dataset', 'cmdosu', 'total', 'cmroute', 'order', 'length', 'tsparmcd', 'lbtestcd',
                            'failures', '%', 'affected', 'records', 'lbtest', 'lborresu', 'egstresu', 'epoch', 'ecdosu',
                            'egorresu', 'egtest', 'egtestcd', 'exdosu', 'lbstresu', '<', 'pcspccnd', 'vsorresu', 'vsstresu',
                            'vstest', 'vstestcd', 'qscat','aeout'
                        ]
                    ])
                ##################################################################################
                print("Title:", title)
                print("Description:", description)
                print("Dataset:",dataset_text)
                print("Filtered Table Header:", filtered_table_header)
                
                # Find the matching row based on title and description
                matching_row = corpus_df[(corpus_df['ID'] == title)] # & (corpus_df['Description'] == description)]
                if not matching_row.empty:
                    print("Matching issue found in corpus, proceeding to Explanation section.")
                                # Navigate to the area containing the explanation button
                    explanation_area = WebDriverWait(driver, 50).until(
                        EC.presence_of_element_located((By.XPATH, '/html/body/div[5]/div/div/div/div[1]/div[1]/div/ul'))
                    )
                    time.sleep(10)
                    # Click on the "Explanation" button
                    explanation_button = WebDriverWait(driver, 50).until(
                        EC.element_to_be_clickable((By.XPATH, '/html/body/div[5]/div/div/div/div[1]/div[1]/div/ul/li[2]'))
                    )
                    explanation_button.click()
                    time.sleep(10)
                # print(matching_row)
                ##################################################adding codes to skip to click on save button when explanation is already added#######################
                    explain_area = WebDriverWait(driver,50).until(
                        EC.presence_of_element_located((By.XPATH,'/html/body/div[5]/div/div/div/div[1]/div[1]/div/div[2]/div/div[1]/div/div[1]/div[2]/div/div/div'))
                    )
                    explain_text = explain_area.find_element(By.XPATH,'/html/body/div[5]/div/div/div/div[1]/div[1]/div/div[2]/div/div[1]/div/div[1]/div[2]/div/div/div/div/div').text
                    print(explain_text)
                    if explain_text =='':
                #######################################################################################################################################
                        def format_filtered_table_header(header):
                            if "TREATMENT" in header:
                                treatment_groups=re.findall(r'TREATMENT \d+',header)
                                return " • ".join(treatment_groups)
                            else:
                                return " • ".join(header.split())
    
                        formatted_header = format_filtered_table_header(filtered_table_header)
                        print(formatted_header)
                        explanation_text = (matching_row.iloc[0, 3].replace('@dataset', dataset_text) .replace('@variable', formatted_header) .replace('@unit', formatted_header))
                                                            
                        # Navigate to the area containing the explanation input
                        explanation_input = WebDriverWait(driver, 100).until(
                            EC.presence_of_element_located((By.XPATH, '/html/body/div[5]/div/div/div/div[1]/div[1]/div/div[2]/div/div[1]/div/div[1]/div[2]/div/div[2]/div/div/div/div'))
                        )
                        explanation_input.send_keys(explanation_text)
                        time.sleep(10)
        
                        save_button = WebDriverWait(driver, 100).until(
                            EC.element_to_be_clickable((By.XPATH, '/html/body/div[5]/div/div/div/div[1]/div[1]/div/div[2]/div/div[1]/div/div[2]/button[1]'))
                        )
                        save_button.click()
    
                                    # Navigate to the area with the next button
                        next_button_area = WebDriverWait(driver, 50).until(
                            EC.presence_of_element_located((By.XPATH, '/html/body/div[5]/div/div/div/div[1]/div[1]/div/ul'))
                        )
                        time.sleep(5)
                        # Click on the issue details tab 
                        issue_details = WebDriverWait(driver, 50).until(
                            EC.element_to_be_clickable((By.XPATH, '/html/body/div[5]/div/div/div/div[1]/div[1]/div/ul/li[1]'))
                        )
                        issue_details.click()
                        time.sleep(10)
    
                        manage_area = WebDriverWait(driver,80).until(
                            EC.presence_of_element_located((By.XPATH,'/html/body/div[5]/div/div/div/div[1]/div[1]/div/div[1]/div/section[2]'))
                        )
                        #Clicking on the status drop down menu
                        drop_down = WebDriverWait(driver,80).until(
                            EC.element_to_be_clickable((By.XPATH,'/html/body/div[5]/div/div/div/div[1]/div[1]/div/div[1]/div/section[2]/aside[1]/div[1]/div/div'))
                        )
                        drop_down.click()
                        time.sleep(10)
                        
                        closed_option = WebDriverWait(driver, 80).until(
                        EC.element_to_be_clickable((By.XPATH, "//div[text()='Closed']"))
                        )
                        closed_option.click() 
                        #naviagte to the manage issues area again #######################################
                        manage_area = WebDriverWait(driver,80).until(
                            EC.presence_of_element_located((By.XPATH,'/html/body/div[5]/div/div/div/div[1]/div[1]/div/div[1]/div/section[2]'))
                        )
                        #edited the code here ###############################################as the assignee name is not coming
                        #Clicking on the status drop down menu
                        drop_down2 = WebDriverWait(driver,80).until(
                            EC.element_to_be_clickable((By.XPATH,'/html/body/div[5]/div/div/div/div[1]/div[1]/div/div[1]/div/section[2]/aside[1]/div[2]/div/div/div[1]'))
                        )
                        drop_down2.click()
                        time.sleep(10)
                        # print(manage_area.get_attribute('outerHTML'))
                        srivastava_option = WebDriverWait(driver, 80).until(
                                EC.element_to_be_clickable((By.XPATH, f"//div[contains(@class, 'react-select__option') and text()='{programmer_name}']"))
                        )
                        srivastava_option.click()
                        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", manage_area)
                        # print("yes")
                    else:
                        print("Explanation text is already present, skipping to the next button.")
                else:
                    print("No matching explanation found in the corpus.")
                    manage_area = WebDriverWait(driver, 80).until(
                        EC.presence_of_element_located((By.XPATH, '/html/body/div[5]/div/div/div/div[1]/div[1]/div/div[1]/div/section[2]'))
                    )
                    driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", manage_area)
                    # if title=="SD0060: Variable in dataset is not present in define.xml" or "user-defined codelist" in title():
                    if "define.xml" in title.lower() or "user-defined codelist" in title.lower() or "sd1015: epoch is not in" in title.lower():
                        # Wait for the comments area to load
                        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", manage_area)
                        comments_area = WebDriverWait(driver, 80).until(
                            EC.presence_of_element_located((By.XPATH, "/html/body/div[5]/div/div/div/div[1]/div[1]/div/div[1]/div/section[2]/aside[2]/div"))
                        )
                        time.sleep(10)  # Optional, consider replacing with an explicit wait if necessary
                        
                        # Determine the comment based on the title
                        
                        ######commented for C5171002 ############################################3
                        if "define.xml" in title.lower() or "user-defined codelist" in title.lower():
                            comment_text = "You have to make the changes in define.xml"
                        elif "sd1015: epoch is not in" in title.lower():
                            comment_text = "You have to edit the Trial Designs.xlsx."
                        
                        # Click on the specific comment section
                        comment_section = WebDriverWait(driver, 80).until(
                            EC.element_to_be_clickable((By.XPATH, "/html/body/div[5]/div/div/div/div[1]/div[1]/div/div[1]/div/section[2]/aside[2]/div/div[2]/div/div[1]/div"))
                        )
                        comment_section.click()
                    
                        # Add the dynamic comment to the comment input field
                        comment_input = WebDriverWait(driver, 80).until(
                            EC.presence_of_element_located((By.XPATH, "//textarea[contains(@class, 'comment-editor__input')]"))  # Update the XPath to match the actual input field
                        )
                        comment_input.clear()  # Clear any existing text, if necessary
                        comment_input.send_keys(comment_text)
                    
                        # Click on the save button
                        time.sleep(10)
                        save_button = WebDriverWait(driver, 80).until(
                            EC.element_to_be_clickable((By.XPATH, "/html/body/div[5]/div/div/div/div[1]/div[1]/div/div[1]/div/section[2]/aside[2]/div/div[2]/div/div[2]/button[1]"))
                        )
                        save_button.click()
    
                ##############################################################################
                ##############################################################################
                    # if "define.xml" in title.lower() or "user-defined codelist" in title.lower() or "sd1015: epoch is not in" in title.lower():
                    # # Wait for the comments area to load
                    #     driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", manage_area)
                    #     comments_area = WebDriverWait(driver, 80).until(
                    #         EC.presence_of_element_located((By.XPATH, "/html/body/div[5]/div/div/div/div[1]/div[1]/div/div[1]/div/section[2]/aside[2]/div"))
                    #     )
                    #     time.sleep(10)  # Optional, consider replacing with an explicit wait if necessary
                    #     # print(1)
                    #     # Click on the specific comment section
                    #     comment_section = WebDriverWait(driver, 80).until(
                    #         EC.element_to_be_clickable((By.XPATH, "/html/body/div[5]/div/div/div/div[1]/div[1]/div/div[1]/div/section[2]/aside[2]/div/div[2]/div/div[1]/div"))
                    #     )
                    #     comment_section.click()
                    #     # print(2)
                    #     # Add the text to the comment input field
                    #     comment_input = WebDriverWait(driver, 80).until(
                    #         EC.presence_of_element_located((By.XPATH, "//textarea[contains(@class, 'comment-editor__input')]"))  # Update the XPath to match the actual input field
                    #     )
                    #     comment_input.clear()  # Clear any existing text, if necessary
                    #     comment_input.send_keys("You have to edit the define.xml")
                    #     # Click on the save button
                    #     time.sleep(10)
                    #     save_button = WebDriverWait(driver, 80).until(
                    #         EC.element_to_be_clickable((By.XPATH, "/html/body/div[5]/div/div/div/div[1]/div[1]/div/div[1]/div/section[2]/aside[2]/div/div[2]/div/div[2]/button[1]"))
                    #     )
                    #     save_button.click()
                    #     # print(1)
                    else:
                        pass
                # else:
                #     print("Explanation text is not empty, proceeding to the next button.")
                
                # Wait for the save operation to complete
                time.sleep(10)
                # print(2)
                # Navigate to the area with the next button
                next_button_area = WebDriverWait(driver, 50).until(
                    EC.presence_of_element_located((By.XPATH, '/html/body/div[5]/div/div/div/div[1]/div[1]/div/ul'))
                )
                time.sleep(5)
                # Click on the issue details tab 
                issue_details = WebDriverWait(driver, 50).until(
                    EC.element_to_be_clickable((By.XPATH, '/html/body/div[5]/div/div/div/div[1]/div[1]/div/ul/li[1]'))
                )
                issue_details.click()
                time.sleep(10)
                
     
                # Click on the button with the specified XPath to go to the next issue
                next_button = WebDriverWait(driver, 50).until(
                    EC.element_to_be_clickable((By.XPATH, '/html/body/div[5]/div/div/div/div[1]/div[1]/div/ul/div/span[3]'))
                )
                next_button.click()
                
            except NoSuchElementException:
                print("No more issues to process or element not found.")
                break
            except TimeoutException as e:
                print(f"Timeout while waiting for an element: {e}")
                break
            except Exception as e:
                print(f"An error occurred: {e}")
                break
    
    finally:
        # driver.quit()
        pass


# Global variables to store file paths
excel_1_path = ""  # Reference Excel file path
excel_2_path = ""  # Study Define Excel file path
#FOR COLOR 
initial_excel_2_copy = None
import shutil
# Function to upload both Excel files (using the existing function)
def upload_excel_files(file_type):
    global excel_1_path, excel_2_path, initial_excel_2_copy
    try:
        if file_type == 'reference':
            excel_1_path = filedialog.askopenfilename(title="Select Reference Excel File", filetypes=[("Excel Files", "*.xlsx;*.xls")])
            if excel_1_path:
                ref_file_label.config(text=f"Reference File: {os.path.basename(excel_1_path)}")
                messagebox.showinfo("Success", "Reference Excel file uploaded successfully!")
                check_all_files_uploaded()  # Check if both files are uploaded
        elif file_type == 'study_define':
            excel_2_path = filedialog.askopenfilename(title="Select Study Define Excel File", filetypes=[("Excel Files", "*.xlsx;*.xls")])
            if excel_2_path:
                study_file_label.config(text=f"Study Define File: {os.path.basename(excel_2_path)}")
                messagebox.showinfo("Success", "Study Define Excel file uploaded successfully!")
                initial_excel_2_copy = f"initial_{os.path.basename(excel_2_path)}"
                shutil.copy(excel_2_path,initial_excel_2_copy)
                check_all_files_uploaded()  # Check if both files are uploaded
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred while uploading the file: {e}")


#########################################FOR 3RD TAB###############################
issues_excel_path =""
study_define_path = ""
def upload_file(file_type):
    global issues_excel_path, study_define_path
    try:
        # Open the file dialog to select the file
        file_path = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx;*.xls"), ("CSV files", "*.csv")])
        
        if file_path:
            # Update the label based on the file type
            if file_type == "issues":
                issues_excel_path = file_path
                issues_excel_file_label.config(text=f"Issues Excel: {file_path.split('/')[-1]}")
            elif file_type == "study_define":
                study_define_path = file_path
                study_define_file_label.config(text=f"Study Define Excel: {file_path.split('/')[-1]}")
            
            # Enable the start button only if both files are uploaded
            if issues_excel_path and study_define_path:
                start_button.config(state=tk.NORMAL)
        else:
            messagebox.showerror("Error", "No file selected.")
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred while uploading the file: {e}")

# Global variables
ref_file_path = None
selected_datasets = []

# Function to upload the reference Excel file and extract unique datasets
def upload_reference_file():
    global ref_file_path, selected_datasets
    
    try:
        # Open file dialog to select the reference Excel file
        ref_file_path = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx;*.xls")])
        
        if ref_file_path:
            ref_file_label.config(text=f"Reference File: {ref_file_path.split('/')[-1]}")  # Update label with file name

            # # Clear any previous checkboxes and dataset selections
            # for widget in datasets_frame.winfo_children():
            #     widget.destroy()
            dataset_listbox.delete(0,tk.END)
            
            try:
                # Read the Datasets sheet from the uploaded reference Excel file
                reference_df = pd.read_excel(ref_file_path, sheet_name="Datasets")
                
                if "Dataset" in reference_df.columns:
                    unique_datasets = reference_df["Dataset"].unique()

                    # Clear the list of selected datasets
                    # selected_datasets = []

                    # Populate the frame with checkboxes for each unique dataset
                    for dataset in unique_datasets:
                        dataset_listbox.insert(tk.END,dataset)

                    # Update the UI with the datasets frame
                    messagebox.showinfo("Success", "Reference file uploaded successfully. Please select datasets.")
                else:
                    messagebox.showerror("Error", "The 'Datasets' sheet is missing the 'Dataset' column.")
            except Exception as e:
                messagebox.showerror("Error", f"An error occurred while reading the reference file: {e}")
        else:
            messagebox.showerror("Error", "No file selected.")
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred while uploading the file: {e}")


# Function to download the selected datasets and update the Excel file
def download_selected_datasets():
    global ref_file_path, selected_datasets
    
    try:
        # Check if a reference file has been uploaded
        if not ref_file_path:
            messagebox.showerror("Error", "Please upload the reference file first.")
            return

        # Filter the selected datasets based on checkboxes
        # selected_dataset_names = [dataset[0] for dataset in selected_datasets if dataset[1].get() == 1]
        selected_dataset_names = [dataset_listbox.get(i) for i in dataset_listbox.curselection()]

        if not selected_dataset_names:
            messagebox.showerror("Error", "Please select at least one dataset.")
            return
        
        # Read the Datasets sheet from the uploaded reference Excel file
        reference_df = pd.read_excel(ref_file_path, sheet_name="Datasets")
        
        # Filter the Datasets sheet to only include selected datasets
        filtered_reference_df = reference_df[reference_df["Dataset"].isin(selected_dataset_names)]

        # Read the other sheets that need to be filtered (Variables, Codelists, Methods, Comments)
        variables_df = pd.read_excel(ref_file_path, sheet_name="Variables")
        codelists_df = pd.read_excel(ref_file_path, sheet_name="Codelists")
        methods_df = pd.read_excel(ref_file_path, sheet_name="Methods")
        comments_df = pd.read_excel(ref_file_path, sheet_name="Comments")
        value_level_df = pd.read_excel(ref_file_path,sheet_name="ValueLevel")

         # Filter the Variables sheet based on the selected datasets
        filtered_variables_df = variables_df[variables_df["Dataset"].isin(selected_dataset_names)]
        #filter the valulevel sheet based on the selected datasets
        filtered_valuelevel_df = value_level_df[value_level_df["Dataset"].isin(selected_dataset_names)]
        # Store the associated Method, Comment, and Codelist for each selected dataset
        selected_methods = filtered_variables_df["Method"].dropna().unique().tolist()
        selected_comments = filtered_variables_df["Comment"].dropna().unique().tolist()
        selected_codelists = filtered_variables_df["Codelist"].dropna().unique().tolist()
        # print(selected_methods)
        # print(selected_comments)
        # Store the associated Method, Comment, and Codelist for each selected dataset in value level
        selected_methods_s = filtered_valuelevel_df["Method"].dropna().unique().tolist()
        selected_comments_s = filtered_valuelevel_df["Comment"].dropna().unique().tolist()
        selected_codelists_s = filtered_valuelevel_df["Codelist"].dropna().unique().tolist()
        
        # Now filter the Methods, Comments, and Codelists sheets based on the selected datasets (using ID column)
        filtered_methods_df = methods_df[methods_df["ID"].isin(selected_methods)]
        filtered_comments_df = comments_df[comments_df["ID"].isin(selected_comments)]
        filtered_codelists_df = codelists_df[codelists_df["ID"].isin(selected_codelists)]

        # Now filter the Methods, Comments, and Codelists sheets based on the selected datasets (using ID column)
        filtered_methods_df_s = methods_df[methods_df["ID"].isin(selected_methods_s)]
        filtered_comments_df_s = comments_df[comments_df["ID"].isin(selected_comments_s)]
        filtered_codelists_df_s = codelists_df[codelists_df["ID"].isin(selected_codelists_s)]

                 # COMBINE METHODS & COMMENTS and valuelevel from variables and value level sheet
        # ==============================
        combined_methods_df = pd.concat([filtered_methods_df, filtered_methods_df_s], ignore_index=True)
        combined_methods_df.drop_duplicates(subset=['ID'], inplace=True)
    
        combined_comments_df = pd.concat([filtered_comments_df, filtered_comments_df_s], ignore_index=True)
        combined_comments_df.drop_duplicates(subset=['ID'], inplace=True)

        combined_codelists_df = pd.concat([filtered_codelists_df, filtered_codelists_df], ignore_index=True)
        combined_codelists_df.drop_duplicates(subset=['ID'], inplace=True)
        
         # Retain the unmodified sheets
        other_sheets = ['Define', 'ValueLevel', 'Dictionaries', 'Documents']  # Add the sheets that should not be changed
        all_sheets = pd.read_excel(ref_file_path, sheet_name=None)  # Read all sheets

        # Update only the modified sheets
        with pd.ExcelWriter(ref_file_path, engine='openpyxl') as writer:
            # Write the filtered data to the existing sheets
            filtered_reference_df.to_excel(writer, sheet_name="Datasets", index=False)
            filtered_variables_df.to_excel(writer, sheet_name="Variables", index=False)
            combined_codelists_df.to_excel(writer, sheet_name="Codelists", index=False)
            combined_methods_df.to_excel(writer, sheet_name="Methods", index=False)
            combined_comments_df.to_excel(writer, sheet_name="Comments", index=False)
            filtered_valuelevel_df.to_excel(writer, sheet_name="ValueLevel", index=False)
            
            # Write unmodified sheets back to the Excel file
            for sheet_name in other_sheets:
                if sheet_name in all_sheets:
                    all_sheets[sheet_name].to_excel(writer, sheet_name=sheet_name, index=False)

        messagebox.showinfo("Success", f"Selected datasets have been saved back to: {ref_file_path}")
    
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred while downloading the datasets: {e}")


# Function to check if both files are uploaded
def check_all_files_uploaded():
    if excel_1_path and excel_2_path:
        start_button.config(state=tk.NORMAL)  # Enable the 'Start Process' button


def highlighted_updated_cells():
    updated_excel_2 = load_workbook(excel_2_path)
    initial_excel_2 = load_workbook(initial_excel_2_copy)
    yellow_fill = PatternFill(start_color="FFFF00",end_color="FFFF00",fill_type="solid")
    #compare all sheets in the initial and updated excel files
    for sheet_name in updated_excel_2.sheetnames:
        initial_sheet = initial_excel_2[sheet_name]
        updated_sheet = updated_excel_2[sheet_name]

        for row in updated_sheet.iter_rows(min_row=2,min_col=1,max_row=updated_sheet.max_row,max_col=updated_sheet.max_column):
            for cell in row:
                initial_cell = initial_sheet.cell(row=cell.row,column=cell.column)
                if initial_cell.value != cell.value:
                    cell.fill = yellow_fill

    updated_excel_2.save(excel_2_path)
    print("cells updated and highlighted in yellow")

                

def start_backend_process():
    # Check if both Excel files are uploaded
    if not excel_1_path or not excel_2_path:
        messagebox.showerror("Error", "Please upload both Excel files first.")
        return

    # Get dataset to delete from entry field
    dataset_to_delete = dataset_delete_entry.get().strip()

    #################22oct2025##############
    dataset_type = dataset_type_dropdown.get()
    terminology_version = terminology_version_dropdown.get()
    standard_version = standard_version_dropdown.get()

    if dataset_type not in ['ADAM','SDTM']:
        messagebox.showerror("Error", "Please select a valid dataset type (ADAM or SDTM).")
        return

    if not terminology_version:
        messagebox.showerror("Error, Please select a terminology version")
        return

    if not standard_version:
        messagebox.showerror("Error, Please select a terminology version")
        return

        ################

    try:
        # Call backend function with all parameters
        edit_excel_files(excel_1_path, excel_2_path,dataset_type,terminology_version, dataset_to_delete,standard_version)
        highlighted_updated_cells()
        messagebox.showinfo("Success", "Backend process completed successfully.")
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred:\n{str(e)}")

def edit_excel_files(excel_1_path,excel_2_path,dataset_type,terminology_version,dataset_to_delete=None,standard_version=None):
    excel_1_path = excel_1_path
    sheet_1_name_variables = "Variables"
    sheet_1_name_methods = "Methods"
    sheet_1_name_comments = "Comments"
    #########new addition for value level #########
    sheet_1_name_valuelevel="ValueLevel"
    ############################
    
    
    excel_2_path = excel_2_path
    sheet_2_name_variables = "Variables"
    #########new addition for value level #########
    sheet_2_name_valuelevel="ValueLevel"
    ############################

    terminology_file = None
    if dataset_type=="ADAM":
        ###start of terminology section for adam part#
        if terminology_version == "ADAM 2025-09-26":
            terminology_file = r"C:\Users\SRIVAA86\Downloads\ADaM_CT_2025-09-26.xlsx"
        elif terminology_version == "ADAM 2025-03-28":
            terminology_file = r"C:\Users\SRIVAA86\Downloads\ADaM_CT_2025-03-28.xlsx"
        elif terminology_version == "ADAM 2024-09-27":
            terminology_file = r"C:\Users\SRIVAA86\Downloads\ADaM_CT_2024-09-27.xlsx"
        else:
            terminology_file = None

        #########to add sdtm/adam version in the datasets sheet
        try:
            study_df = pd.read_excel(excel_2_path,sheet_name="Datasets")

            if standard_version:
                study_df['Standard'] = standard_version

            # with pd.ExcelWriter(excel_2_path,engine='openpyxl',mode='a') as writer:
            #     study_df.to_excel(writer,sheet_name="Datasets",index=False)

        except Exception as e:
            messagebox.showerror("Error",f"An error occured while editing the excel file: {e}")
            return

        try:
            terminology_df = pd.read_excel(terminology_file,sheet_name=pd.ExcelFile(terminology_file).sheet_names[1])
            df3_variables = pd.read_excel(excel_2_path,sheet_name="Codelists")

            terminology_df['Code'] = terminology_df['Code'].astype(str)
            # df3_variables['NCI Term Code'] = df3_variables['NCI Term Code'].astype(str)
            df3_variables['NCI Codelist Code'] = df3_variables['NCI Codelist Code'].apply(lambda x: '' if pd.isna(x) else x)
            # Update 'Terminology' in df3_variables based on matching codes
            for index, row in terminology_df.iterrows():
                matching_code = row['Code']
                matching_rows = df3_variables[df3_variables['NCI Codelist Code'] == matching_code]
    
            if not matching_rows.empty:
                df3_variables.loc[df3_variables["NCI Codelist Code"] == matching_code, 'Terminology'] = terminology_version

        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {e}")
            return
##########end of terminology section for adam part ############
        
        # Step 1: Read the 'Variables' sheets from both Excel files
        df1_variables = pd.read_excel(excel_1_path, sheet_name=sheet_1_name_variables)
        df2_variables = pd.read_excel(excel_2_path, sheet_name=sheet_2_name_variables)
        
        # Merge the DataFrames based on common columns
        merge_columns = ['Dataset', 'Variable']
        merged_df = pd.merge(
            df2_variables,
            df1_variables[['Variable', 'Dataset', 'Comment', 'Origin', 'Source', 'Method', 'Predecessor', 'Data Type']],
            how='left',
            on=merge_columns
        )
        
        # Update columns with values from df1 only if df2 entries are blank
        columns_to_update = ['Comment', 'Origin', 'Source', 'Method', 'Predecessor', 'Data Type']
        for column in columns_to_update:
            merged_df[column] = merged_df[column + '_x'].combine_first(merged_df[column + '_y'])
            merged_df.drop(columns=[f'{column}_x', f'{column}_y'], inplace=True)
            
        # Sort by 'Dataset' and 'Order'
        sort_columns = ['Dataset', 'Order']
        sorted_df = merged_df.sort_values(by=sort_columns)
        
        # Replace 0 with 1 in the 'Length' column
        sorted_df['Length'].replace(0, 1, inplace=True)
        
        # Update 'Source' where 'Origin' is 'assigned' or 'derived'
        sorted_df['Source'] = np.where(
            sorted_df['Origin'].str.lower().isin(['assigned', 'derived']),
            'Sponsor',
            ''
        )
        
        # Handle 'Has No Data' and 'Comment' columns
        sorted_df['Has No Data'].fillna('', inplace=True)
        sorted_df['Comment'] = np.where(
            sorted_df['Has No Data'].str.strip().str.lower() == 'yes',
            sorted_df['Comment'],  # Retain original comment if 'yes'
            ''  # Otherwise, set to empty
        )
        
        # Step 2: Filter Methods based on the 'Method' column from the variables sheet
        df1_methods = pd.read_excel(excel_1_path, sheet_name=sheet_1_name_methods)
        methods_list = sorted_df['Method'].dropna().tolist()
        filtered_methods_df = df1_methods[df1_methods['ID'].isin(methods_list)]
        # print(filtered_methods_df)
        
        # Step 3: Filter Comments based on the 'Comment' column from the variables sheet
        df1_comments = pd.read_excel(excel_1_path, sheet_name=sheet_1_name_comments)
        comments_list = sorted_df['Comment'].dropna().tolist()
        filtered_comments_df = df1_comments[df1_comments['ID'].isin(comments_list)]
    
        ##########################new addition for value level #######################
        try:
            df1_valuelevel = pd.read_excel(excel_1_path, sheet_name=sheet_1_name_valuelevel)
            df2_valuelevel = pd.read_excel(excel_2_path,sheet_name=sheet_2_name_valuelevel)
    
            #merge based on dataset + variable + where clause variable 
            merge_columns_v1 = ['Dataset','Variable','Where Clause']
            merged_v1_df = pd.merge(
                df2_valuelevel,
                df1_valuelevel[['Dataset','Variable','Where Clause','Data Type',
                                'Length','Origin','Source','Method','Predecessor',
                                'Comment'
                ]],
                how='left',
                on=merge_columns_v1
            )
    
            #update only if blank
            columns_to_update_v1 = ['Data Type','Length','Origin','Source','Method','Predecessor','Comment']
            for column in columns_to_update_v1:
                merged_v1_df[column] = merged_v1_df[column + '_x'].combine_first(merged_v1_df[column + '_y'])
                merged_v1_df.drop(columns=[f'{column}_x', f'{column}_y'],inplace=True)
    
            sorted_v1_df = merged_v1_df.sort_values(by=['Dataset', 'Variable', 'Where Clause'])
            #replace 0 with 1 in length
            sorted_v1_df['Length'].replace(0,1,inplace=True)
            # Filter Methods
            methods_list_v1 = sorted_v1_df['Method'].dropna().tolist()
            filtered_methods_v1_df = df1_methods[df1_methods['ID'].isin(methods_list_v1)]
            # print(filtered_methods_v1_df)
    
            # Filter Comments
            comments_list_v1 = sorted_v1_df['Comment'].dropna().tolist()
            filtered_comments_v1_df = df1_comments[df1_comments['ID'].isin(comments_list_v1)]
            print("ValueLevel sheet merged successfully.")
        except Exception as e:
            print("ValueLevel sheet not found in one of the files or error occurred:", e)
            sorted_v1_df = pd.DataFrame() 
            filtered_methods_v1_df = pd.DataFrame()
            filtered_comments_v1_df = pd.DataFrame()
    
        ############################
    
         # COMBINE METHODS & COMMENTS from variables and value level sheet
        # ==============================
        combined_methods_df = pd.concat([filtered_methods_df, filtered_methods_v1_df], ignore_index=True)
        combined_methods_df.drop_duplicates(subset=['ID'], inplace=True)
    
        combined_comments_df = pd.concat([filtered_comments_df, filtered_comments_v1_df], ignore_index=True)
        combined_comments_df.drop_duplicates(subset=['ID'], inplace=True)
        #################################################################
        
        # Load the workbook into memory
        wb = load_workbook(excel_2_path)
        
        # List of sheets you want to remove
        sheets_to_delete = ['Datasets','Variables', 'Methods', 'Comments','ValueLevel','Codelists']
        
        # Loop through each sheet name
        for sheet_name in sheets_to_delete:
            if sheet_name in wb.sheetnames:  # Check if the sheet exists
                del wb[sheet_name]            # Delete the sheet
                print(f"Deleted sheet: {sheet_name}")
            else:
                print(f"Sheet '{sheet_name}' not found, skipping.")
        
        # Save the workbook after deletions
        wb.save(excel_2_path)
        
           # Write back combined data
        with pd.ExcelWriter(excel_2_path, engine='openpyxl', mode='a') as writer:
            sorted_df.to_excel(writer, sheet_name='Variables', index=False)
            sorted_v1_df.to_excel(writer, sheet_name='ValueLevel', index=False)
            combined_methods_df.to_excel(writer, sheet_name='Methods', index=False)
            combined_comments_df.to_excel(writer, sheet_name='Comments', index=False)
             #####################
            df3_variables.to_excel(writer,sheet_name="Codelists",index=False)
            ###################
            study_df.to_excel(writer,sheet_name="Datasets",index=False)
        
    
            ##############################################
        
        print("DataFrames written to the Excel file successfully.")
        ###
                # ========== DATASET DELETION (SAFE POST-UPDATE) =========
        # Split the dataset names by commas and remove any extra spaces around them
        domain_to_delete = dataset_to_delete.strip() if dataset_to_delete else ""
        datasets_to_delete = [dataset.strip() for dataset in domain_to_delete.split(",") if dataset.strip()]
        
        # Check if there are any datasets to delete
        if datasets_to_delete:
            print(f"Deleting datasets {datasets_to_delete} from all sheets...")
            
            # Read the updated sheets from Excel
            df_variables = pd.read_excel(excel_2_path, sheet_name='Variables')
            df_methods = pd.read_excel(excel_2_path, sheet_name='Methods')
            df_comments = pd.read_excel(excel_2_path, sheet_name='Comments')
            df_codelists = pd.read_excel(excel_2_path,sheet_name='Codelists')
            df_datasets = pd.read_excel(excel_2_path,sheet_name='Datasets')
        
            # Loop through each dataset to delete it
            for domain_to_delete in datasets_to_delete:
                if domain_to_delete:  # Ensure that the dataset name is not empty
                    print(f"Deleting dataset '{domain_to_delete}' from all sheets...")
                    
                    # Filter out the dataset from the 'Variables' sheet
                    to_delete_df = df_variables[df_variables['Dataset'] == domain_to_delete]
                    methods_to_delete = to_delete_df['Method'].dropna().unique().tolist()
                    comments_to_delete = to_delete_df['Comment'].dropna().unique().tolist()
                    codelists_to_delete = to_delete_df['Codelist'].dropna().unique().tolist()
        
                    # Remove the dataset from Variables, Methods, and Comments sheets
                    df_variables = df_variables[df_variables['Dataset'] != domain_to_delete]
                    df_methods = df_methods[~df_methods['ID'].isin(methods_to_delete)]
                    df_comments = df_comments[~df_comments['ID'].isin(comments_to_delete)]
                    df_codelists = df_codelists[~df_codelists['ID'].isin(codelists_to_delete)]
                    df_datasets = df_datasets[df_datasets['Dataset'] != domain_to_delete]
                    
                    print(f"✅ Dataset '{domain_to_delete}' deleted successfully.")
        
            # Overwrite the final Excel file safely after deletions
            wb = load_workbook(excel_2_path)
            for sname in ['Variables', 'Methods', 'Comments','Codelists','Datasets']:
                if sname in wb.sheetnames:
                    del wb[sname]  
            wb.save(excel_2_path)
        
            # Write the updated DataFrames back into the Excel file
            with pd.ExcelWriter(excel_2_path, engine='openpyxl', mode='a') as writer:
                df_variables.to_excel(writer, sheet_name='Variables', index=False)
                df_methods.to_excel(writer, sheet_name='Methods', index=False)
                df_comments.to_excel(writer, sheet_name='Comments', index=False)
                df_codelists.to_excel(writer,sheet_name='Codelists',index=False)
                df_datasets.to_excel(writer,sheet_name='Datasets',index=False)
        
            print(f"✅ All specified datasets deleted and Excel file updated successfully.")
        else:
            print("No datasets specified — skipping deletion step.")
        messagebox.showinfo("Success","Excel file updated successfully")
        
#########################################################SDTM##########################################################################################
    else:
        ###start of terminolgy section#################
        if terminology_version == "SDTM 2025-09-26":
            terminology_file = r"C:\Users\SRIVAA86\Downloads\SDTM_CT_2025-09-26.xlsx"
        elif terminology_version == "SDTM 2025-03-28":
            terminology_file = r"C:\Users\SRIVAA86\Downloads\SDTM_CT_2025-03-28.xlsx"
        elif terminology_version == "SDTM 2024-09-27":
            terminology_file = r"C:\Users\SRIVAA86\Downloads\SDTM_CT_2024-09-27.xlsx"
        else:
            terminology_file=None
            
        try:
            study_df = pd.read_excel(excel_2_path,sheet_name="Datasets")

            if standard_version:
                study_df['Standard'] = standard_version

            # with pd.ExcelWriter(excel_2_path,engine='openpyxl',mode='a') as writer:
            #     study_df.to_excel(writer,sheet_name="Datasets",index=False)

        except Exception as e:
            messagebox.showerror("Error",f"An error occured while editing the excel file: {e}")
            return
            
        try:
            terminology_df = pd.read_excel(terminology_file,sheet_name=pd.ExcelFile(terminology_file).sheet_names[1])
            df3_variables = pd.read_excel(excel_2_path,sheet_name="Codelists")

            terminology_df['Code'] = terminology_df['Code'].astype(str)
            # df3_variables['NCI Term Code'] = df3_variables['NCI Term Code'].astype(str)
            df3_variables['NCI Codelist Code'] = df3_variables['NCI Codelist Code'].apply(lambda x: '' if pd.isna(x) else x)
            # Update 'Terminology' in df3_variables based on matching codes
            for index, row in terminology_df.iterrows():
                matching_code = row['Code']
                matching_rows = df3_variables[df3_variables['NCI Codelist Code'] == matching_code]
    
                if not matching_rows.empty:
                    df3_variables.loc[df3_variables["NCI Codelist Code"] == matching_code, 'Terminology'] = terminology_version

        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {e}")
            return

########################################end of terminology section ########################################
         # Step 1: Read the 'Variables' sheets from both Excel files
        df1_variables = pd.read_excel(excel_1_path, sheet_name=sheet_1_name_variables)
        df2_variables = pd.read_excel(excel_2_path, sheet_name=sheet_2_name_variables)
        
        # Merge the DataFrames based on common columns
        merge_columns = ['Dataset', 'Variable']
        merged_df = pd.merge(
            df2_variables,
            df1_variables[['Variable', 'Dataset', 'Comment', 'Origin', 'Source', 'Method','Data Type']],
            how='left',
            on=merge_columns
        )
        
        # Update columns with values from df1 only if df2 entries are blank
        columns_to_update = ['Comment', 'Origin', 'Source', 'Method','Data Type']
        for column in columns_to_update:
            merged_df[column] = merged_df[column + '_x'].combine_first(merged_df[column + '_y'])
            merged_df.drop(columns=[f'{column}_x', f'{column}_y'], inplace=True)

        # Sort by 'Dataset' and 'Order'
        sort_columns = ['Dataset', 'Order']
        sorted_df = merged_df.sort_values(by=sort_columns)
        
        # Replace 0 with 1 in the 'Length' column
        sorted_df['Length'].replace(0, 1, inplace=True)
        
        # Update 'Source' where 'Origin' is 'assigned' or 'derived'
        sorted_df['Source'] = np.where(
            sorted_df['Origin'].str.lower().isin(['assigned', 'derived']),
            'Sponsor',
            ''
        )
        
        # Handle 'Has No Data' and 'Comment' columns
        sorted_df['Has No Data'].fillna('', inplace=True)
        sorted_df['Comment'] = np.where(
            sorted_df['Has No Data'].str.strip().str.lower() == 'yes',
            sorted_df['Comment'],  # Retain original comment if 'yes'
            ''  # Otherwise, set to empty
        )
        
        # Step 2: Filter Methods based on the 'Method' column from the variables sheet
        df1_methods = pd.read_excel(excel_1_path, sheet_name=sheet_1_name_methods)
        methods_list = sorted_df['Method'].dropna().tolist()
        filtered_methods_df = df1_methods[df1_methods['ID'].isin(methods_list)]
        # print(filtered_methods_df)
        
        # Step 3: Filter Comments based on the 'Comment' column from the variables sheet
        df1_comments = pd.read_excel(excel_1_path, sheet_name=sheet_1_name_comments)
        comments_list = sorted_df['Comment'].dropna().tolist()
        filtered_comments_df = df1_comments[df1_comments['ID'].isin(comments_list)]
    
        ##########################new addition for value level #######################
        try:
            df1_valuelevel = pd.read_excel(excel_1_path, sheet_name=sheet_1_name_valuelevel)
            df2_valuelevel = pd.read_excel(excel_2_path,sheet_name=sheet_2_name_valuelevel)
    
            #merge based on dataset + variable + where clause variable 
            merge_columns_v1 = ['Dataset','Variable','Where Clause']
            merged_v1_df = pd.merge(
                df2_valuelevel,
                df1_valuelevel[['Dataset','Variable','Where Clause','Data Type',
                                'Length','Origin','Source','Method',
                                'Comment'
                ]],
                how='left',
                on=merge_columns_v1
            )
    
            #update only if blank
            columns_to_update_v1 = ['Data Type','Length','Origin','Source','Method','Comment']
            for column in columns_to_update_v1:
                merged_v1_df[column] = merged_v1_df[column + '_x'].combine_first(merged_v1_df[column + '_y'])
                merged_v1_df.drop(columns=[f'{column}_x', f'{column}_y'],inplace=True)
    
            sorted_v1_df = merged_v1_df.sort_values(by=['Dataset', 'Variable', 'Where Clause'])
            #replace 0 with 1 in length
            sorted_v1_df['Length'].replace(0,1,inplace=True)
            # Filter Methods
            methods_list_v1 = sorted_v1_df['Method'].dropna().tolist()
            filtered_methods_v1_df = df1_methods[df1_methods['ID'].isin(methods_list_v1)]
            # print(filtered_methods_v1_df)
    
            # Filter Comments
            comments_list_v1 = sorted_v1_df['Comment'].dropna().tolist()
            filtered_comments_v1_df = df1_comments[df1_comments['ID'].isin(comments_list_v1)]
            print("ValueLevel sheet merged successfully.")
        except Exception as e:
            print("ValueLevel sheet not found in one of the files or error occurred:", e)
            sorted_v1_df = pd.DataFrame() 
            filtered_methods_v1_df = pd.DataFrame()
            filtered_comments_v1_df = pd.DataFrame()
    
        ############################
    
         # COMBINE METHODS & COMMENTS from variables and value level sheet
        # ==============================
        combined_methods_df = pd.concat([filtered_methods_df, filtered_methods_v1_df], ignore_index=True)
        combined_methods_df.drop_duplicates(subset=['ID'], inplace=True)
    
        combined_comments_df = pd.concat([filtered_comments_df, filtered_comments_v1_df], ignore_index=True)
        combined_comments_df.drop_duplicates(subset=['ID'], inplace=True)
        
        #################################################################
        
        # Load the workbook into memory
        wb = load_workbook(excel_2_path)
        
        # List of sheets you want to remove
        sheets_to_delete = ['Datasets','Variables', 'Methods', 'Comments','ValueLevel','Codelists']
        
        # Loop through each sheet name
        for sheet_name in sheets_to_delete:
            if sheet_name in wb.sheetnames:  # Check if the sheet exists
                del wb[sheet_name]            # Delete the sheet
                print(f"Deleted sheet: {sheet_name}")
            else:
                print(f"Sheet '{sheet_name}' not found, skipping.")
        
        # Save the workbook after deletions
        wb.save(excel_2_path)
        
           # Write back combined data
        with pd.ExcelWriter(excel_2_path, engine='openpyxl', mode='a') as writer:
            sorted_df.to_excel(writer, sheet_name='Variables', index=False)
            sorted_v1_df.to_excel(writer, sheet_name='ValueLevel', index=False)
            combined_methods_df.to_excel(writer, sheet_name='Methods', index=False)
            combined_comments_df.to_excel(writer, sheet_name='Comments', index=False)
            #####################
            df3_variables.to_excel(writer,sheet_name="Codelists",index=False)
            ###################
            study_df.to_excel(writer,sheet_name="Datasets",index=False)
        
    
            ##############################################
        
        print("DataFrames written to the Excel file successfully.")
        ###
        # ========== DATASET DELETION (SAFE POST-UPDATE) =========
        # Split the dataset names by commas and remove any extra spaces around them
        domain_to_delete = dataset_to_delete.strip() if dataset_to_delete else ""
        datasets_to_delete = [dataset.strip() for dataset in domain_to_delete.split(",") if dataset.strip()]
        
        # Check if there are any datasets to delete
        if datasets_to_delete:
            print(f"Deleting datasets {datasets_to_delete} from all sheets...")
            
            # Read the updated sheets from Excel
            df_variables = pd.read_excel(excel_2_path, sheet_name='Variables')
            df_methods = pd.read_excel(excel_2_path, sheet_name='Methods')
            df_comments = pd.read_excel(excel_2_path, sheet_name='Comments')
            df_codelists = pd.read_excel(excel_2_path,sheet_name='Codelists')
            df_datasets = pd.read_excel(excel_2_path,sheet_name='Datasets')
        
            # Loop through each dataset to delete it
            for domain_to_delete in datasets_to_delete:
                if domain_to_delete:  # Ensure that the dataset name is not empty
                    print(f"Deleting dataset '{domain_to_delete}' from all sheets...")
                    
                    # Filter out the dataset from the 'Variables' sheet
                    to_delete_df = df_variables[df_variables['Dataset'] == domain_to_delete]
                    methods_to_delete = to_delete_df['Method'].dropna().unique().tolist()
                    comments_to_delete = to_delete_df['Comment'].dropna().unique().tolist()
                    codelists_to_delete = to_delete_df['Codelist'].dropna().unique().tolist()
        
                    # Remove the dataset from Variables, Methods, and Comments sheets
                    df_variables = df_variables[df_variables['Dataset'] != domain_to_delete]
                    df_methods = df_methods[~df_methods['ID'].isin(methods_to_delete)]
                    df_comments = df_comments[~df_comments['ID'].isin(comments_to_delete)]
                    df_codelists = df_codelists[~df_codelists['ID'].isin(codelists_to_delete)]
                    df_datasets = df_datasets[df_datasets['Dataset'] != domain_to_delete]
                    
                    print(f"✅ Dataset '{domain_to_delete}' deleted successfully.")
        
            # Overwrite the final Excel file safely after deletions
            wb = load_workbook(excel_2_path)
            for sname in ['Variables', 'Methods', 'Comments','Codelists','Datasets']:
                if sname in wb.sheetnames:
                    del wb[sname]  
            wb.save(excel_2_path)
        
            # Write the updated DataFrames back into the Excel file
            with pd.ExcelWriter(excel_2_path, engine='openpyxl', mode='a') as writer:
                df_variables.to_excel(writer, sheet_name='Variables', index=False)
                df_methods.to_excel(writer, sheet_name='Methods', index=False)
                df_comments.to_excel(writer, sheet_name='Comments', index=False)
                df_codelists.to_excel(writer,sheet_name='Codelists',index=False)
                df_datasets.to_excel(writer,sheet_name='Datasets',index=False)
        
            print(f"✅ All specified datasets deleted and Excel file updated successfully.")
        else:
            print("No datasets specified — skipping deletion step.")
        messagebox.showinfo("Success","Excel file updated successfully")


###########################to add the 3rd tab#############################

model = BartForSequenceClassification.from_pretrained(r"C:\Users\SRIVAA86\Downloads\saved_model")
tokenizer = BartTokenizer.from_pretrained(r"C:\Users\SRIVAA86\Downloads\saved_model")

def predict_comment(comment):
    inputs = tokenizer(comment, return_tensors="pt", padding=True, truncation=True, max_length=128)
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_label = np.argmax(logits.detach().numpy())
    return label_to_description[predicted_label]


def start_the_process():
    if not issues_excel_path or not study_define_path:
        messagebox.showerror("Error","Please Upload both Issues Excel and study define excel files")
        return
    try:
        issues_df = read_issues(issues_excel_path)
        fix_issue_in_sheet(study_define_path,issues_df)
    except Exception as e:
        messagebox.showerror("Error",f"An error occured: {e}")
    
def read_issues(sheet_path):
    try:
        issues_df = pd.read_csv(sheet_path)
        issues_df[['Variables_A','Variables_B']] = issues_df['Variables'].str.split(',',n=1,expand=True)
        issues_df['Variables_A']=issues_df['Variables_A'].fillna('')
        issues_df['Variables_B']=issues_df['Variables_B'].fillna('')
        issues_df['Variables_B']=issues_df['Variables_B'].apply(lambda x: 'Codelists' if 'CodeList Name' in x else x)

        issues_df[['Values_A','Values_B']] = issues_df['Values'].str.split(',',n=1,expand=True)
        issues_df['Values_A']=issues_df['Values_A'].fillna('')
        issues_df['Values_B']=issues_df['Values_B'].fillna('')
        return issues_df
    except Exception as e:
        print(f"Error reading issues: {e}")
        return None

def fix_issue_in_sheet(study_define_path,issues_df):
    try:
        for index, row in issues_df.iterrows():
            print(f"checking row {index}: Variables_B = '{row['Variables_B']}'")
            if "Codelists" in row["Variables_B"]:
                sheet_name = "Codelists"
                study_define_df=pd.read_excel(study_define_path,sheet_name=sheet_name)
                match_row = study_define_df[study_define_df['ID'] == row['Values_A']]

                if not match_row.empty:
                    value = row['Values_A'].split('.')[1] if row['Values_A'] else ''
                    current_name = match_row.iloc[0]['Name']
                    new_name = f"{current_name} for {value}" if value else current_name
                    print(new_name)
                    study_define_df.loc[study_define_df['ID'] == row['Values_A'],'Name'] = new_name
    
                    with pd.ExcelWriter(study_define_path,engine='openpyxl',mode='a',if_sheet_exists='replace') as writer:
                        study_define_df.to_excel(writer,sheet_name=sheet_name,index=False)
    
                print("All issues fixed successfully")
            if row["Variables_B"] == " Variable":
                sheet_name = "Variables"
                study_define_df = pd.read_excel(study_define_path,sheet_name=sheet_name)
                
                #find the row where both values_A(dataset) and values_b(variable) match
                match_row = study_define_df[(study_define_df['Dataset'] == row['Values_A'].strip()) & (study_define_df['Variable'] == row['Values_B'].strip())] 
                print(match_row)
                if not match_row.empty:
                    comment=f"{row['Values_A']}.{row['Values_B'].strip()}"
                    print(comment)
                    study_define_df.loc[(study_define_df['Dataset'] == row['Values_A'].strip()) & (study_define_df['Variable'] == row['Values_B'].strip()),'Comment'] = comment
                    description = predict_comment(comment) #the BART model 
                    print(description)
                    with pd.ExcelWriter(study_define_path,engine='openpyxl',mode='a',if_sheet_exists='replace') as writer:
                        study_define_df.to_excel(writer,sheet_name=sheet_name,index=False)

                    #now we need to update the comment sheet
                    comment_sheet="Comments"
                    comment_df = pd.read_excel(study_define_path,sheet_name=comment_sheet)

                    new_comment_row = pd.DataFrame({
                        'ID' : [comment],
                        'Description' : [description]
                    })

                    # append the new comment row to the comment sheet
                    comment_df = pd.concat([comment_df,new_comment_row],ignore_index=True)

                    with pd.ExcelWriter(study_define_path,engine='openpyxl',mode='a',if_sheet_exists='replace') as writer:
                        comment_df.to_excel(writer,sheet_name=comment_sheet,index=False)

            # else:
            #     pass
        print("Issues fixed")
    except Exception as e:
        print(f"Error fixing issue: {e}")

def create_gui():
    global dataset_listbox, ref_file_labe
    root = tk.Tk()
    root.title("Pinnacle 21 Automation Tool")

    # Set the overall font style for the application
    app_font = ("Helvetica", 12)

    # Set the style for ttk widgets
    style = ttk.Style()
    style.configure("TNotebook.Tab", background="#007BFF", font=("Helvetica", 12, "bold"), foreground="black")
    style.configure("TButton", font=app_font, padding=10)
    style.configure("TLabel", font=app_font, background="white",foreground="black")
    style.configure("TCombobox", font=app_font, background="white")

    # Header frame for the title
    header_frame = tk.Frame(root, bg="#007BFF", height=50)
    header_frame.pack(fill="x", padx=0, pady=0)
    
    header_label = tk.Label(header_frame, text="Pinnacle 21 Automation Tool", font=("Helvetica", 16, "bold"), bg="#007BFF", fg="white")
    header_label.pack(expand=True, fill="both", padx=20, pady=20)

    # Tab control
    tab_control = ttk.Notebook(root)
    tab_1 = ttk.Frame(tab_control)
    tab_control.add(tab_1, text="Dataset Selection & Download")
    # tab_4.config(bg="white")

    ref_labe = tk.Label(tab_1, text="Upload Reference Excel file:", font=app_font, bg="white")
    ref_labe.pack(pady=10)

    ref_butto = tk.Button(tab_1, text="Upload Reference File", command=upload_reference_file, font=app_font, bg="#007BFF", fg="white", relief="raised")
    ref_butto.pack(pady=10)

    global ref_file_labe
    ref_file_labe = tk.Label(tab_1, text="Reference File: Not uploaded", font=app_font, bg="white", anchor="w", width=40)
    ref_file_labe.pack(pady=10)

    dataset_label = tk.Label(tab_1, text="Select Datasets:", font=app_font, bg="white")
    dataset_label.pack(pady=10)

    dataset_listbox = tk.Listbox(tab_1, selectmode=tk.MULTIPLE, font=app_font, height=10, width=40)
    dataset_listbox.pack(pady=10)

    download_button = tk.Button(tab_1, text="Download Selected Datasets", command=download_selected_datasets, font=app_font, bg="#4CAF50", fg="white", relief="raised")
    download_button.pack(pady=10)
    

    # Tab 2: Excel Files Upload and Editing
    tab_2 = ttk.Frame(tab_control)
    tab_control.add(tab_2, text="Define MetaData Updates (Excel)")
    # tab_2.config(bg="white")
    global dataset_type_dropdown
    dataset_type_label = tk.Label(tab_2, text="Select Dataset type (SDTM/ADAM):", font=app_font, bg="white")
    dataset_type_label.pack(pady=10)

    dataset_type = tk.StringVar()
    dataset_type_dropdown = ttk.Combobox(tab_2, textvariable=dataset_type, values=["SDTM", "ADAM"], state="readonly", font=("Helvetica", 11), width=20)
    dataset_type_dropdown.pack(pady=5)
    dataset_type_dropdown.set("SDTM")

    terminology_version_label = tk.Label(tab_2, text="Select Terminology Version:", font=app_font, bg="white")
    terminology_version_label.pack(pady=5)
    
    global terminology_version_dropdown
    terminology_version_dropdown = ttk.Combobox(tab_2, state="readonly", font=("Helvetica", 9), width=10)
    terminology_version_dropdown.pack(pady=5)
    
    standard_version_label = tk.Label(tab_2, text="Select Standard Version:", font=("Helvetica", 9), bg="white")
    standard_version_label.pack(pady=5)
    
    global standard_version_dropdown
    standard_version_dropdown = ttk.Combobox(tab_2, state="readonly", font=("Helvetica", 11), width=10)
    standard_version_dropdown.pack(pady=5)
    
    def update_versions(event=None):
        dataset_type_selected = dataset_type.get()
        if dataset_type_selected == "SDTM":
            terminology_version_dropdown['values'] = ["SDTM 2025-09-26", "SDTM 2025-03-28", "SDTM 2024-09-27"]
            standard_version_dropdown['values'] = ["SDTM IG 3.4", "SDTM IG 3.3", "SDTM IG 3.2"]
        elif dataset_type_selected == "ADAM":
            terminology_version_dropdown['values'] = ["ADAM 2025-09-26", "ADAM 2025-03-28", "ADAM 2024-09-27"]
            standard_version_dropdown['values'] = ["ADAM IG 1.3", "ADAM IG 1.2", "ADAM IG 1.1"]
    
        terminology_version_dropdown.set('SDTM 2025-09-26')
        standard_version_dropdown.set('SDTM IG 3.4')
    
    dataset_type_dropdown.bind("<<ComboboxSelected>>", update_versions)

    ref_label = tk.Label(tab_2, text="Upload Reference Excel file:", font=app_font, bg="white")
    ref_label.pack(pady=10)

    ref_button = tk.Button(tab_2, text="Upload Reference File", command=lambda: upload_excel_files('reference'), font=app_font, bg="#007BFF", fg="white", relief="raised")
    ref_button.pack(pady=10)

    global ref_file_label
    ref_file_label = tk.Label(tab_2, text="Reference File: Not uploaded", font=app_font, bg="white", anchor="w", width=40)
    ref_file_label.pack(pady=10)

    study_label = tk.Label(tab_2, text="Upload Study Define Excel file:", font=app_font, bg="white")
    study_label.pack(pady=10)

    study_button = tk.Button(tab_2, text="Upload Study Define File", command=lambda: upload_excel_files('study_define'), font=app_font, bg="#007BFF", fg="white", relief="raised")
    study_button.pack(pady=10)

    global study_file_label
    study_file_label = tk.Label(tab_2, text="Study Define File: Not uploaded", font=app_font, bg="white", anchor="w", width=40)
    study_file_label.pack(pady=10)

    # Delete Dataset Option
    dataset_label = tk.Label(tab_2, text="Enter Dataset Name to Delete:", font=app_font, bg="white")
    dataset_label.pack(pady=10)

    global dataset_delete_entry
    dataset_delete_entry = tk.Entry(tab_2, width=40, font=app_font)
    dataset_delete_entry.pack(pady=5)

    delete_hint = tk.Label(tab_2, text="(Optional) Leave blank if you don't want to delete any dataset.", font=("Helvetica", 8), fg="gray", bg="#ADD8E6")
    delete_hint.pack(pady=3)

    global start_button
    start_button = tk.Button(tab_2, text="Start Backend Process", command=start_backend_process, font=app_font, bg="#4CAF50", fg="white", relief="raised", state=tk.NORMAL)
    start_button.pack(pady=10)
    
    # Tab 3: ISSUES FIXING
    tab_3 = ttk.Frame(tab_control)
    tab_control.add(tab_3, text="Issues Fixing")
    # tab_3.config(bg="white")

    issues_excel_label = tk.Label(tab_3, text="Upload Issues Excel (CSV):", font=app_font, bg="white")
    issues_excel_label.pack(pady=10)
    
    issues_excel_button = tk.Button(tab_3, text="Upload Issues Excel", command=lambda: upload_file('issues'), font=app_font, bg="#007BFF", fg="white", relief="raised")
    issues_excel_button.pack(pady=10)
    
    global issues_excel_file_label
    issues_excel_file_label = tk.Label(tab_3, text="Issues Excel: Not uploaded", font=app_font, anchor="w", width=40, bg="white")
    issues_excel_file_label.pack(pady=10)
    
    study_define_label = tk.Label(tab_3, text="Upload Study Define Excel:", font=app_font, bg="white")
    study_define_label.pack(pady=10)
    
    study_define_button = tk.Button(tab_3, text="Upload Study Define Excel", command=lambda: upload_file('study_define'), font=app_font, bg="#007BFF", fg="white", relief="raised")
    study_define_button.pack(pady=10)
    
    global study_define_file_label
    study_define_file_label = tk.Label(tab_3, text="Study Define Excel: Not uploaded", font=app_font, anchor="w", width=40, bg="white")
    study_define_file_label.pack(pady=10)
    
    start_button = tk.Button(tab_3, text="Start Fixing Issues", command=start_the_process, font=app_font, bg="#4CAF50", fg="white", relief="raised", state=tk.DISABLED)
    start_button.pack(pady=10)

    # Tab 1: Issues Automation
    tab_4 = ttk.Frame(tab_control)
    tab_control.add(tab_4, text="Issues Automation")
    # tab_1.config(bg="white")  # White background for the tab

    study_name_label = tk.Label(tab_4, text="Enter Study Name:", font=app_font, bg="white")
    study_name_label.pack(pady=10)

    study_name_entry = tk.Entry(tab_4, width=40, font=app_font)
    study_name_entry.pack(pady=10)

    programmer_name_label = tk.Label(tab_4, text="Programmer Name:", font=app_font, bg="white")
    programmer_name_label.pack(pady=10)
    
    programmer_name_entry = tk.Entry(tab_4, width=40, font=app_font)
    programmer_name_entry.pack(pady=10)
    
    def on_run_selenium():
        study_name = study_name_entry.get()
        programmer_name = programmer_name_entry.get()
        if study_name and programmer_name:
            run_selenium_automation(study_name, programmer_name)
        else:
            messagebox.showerror("Error", "Please enter the correct details.")

    run_button = tk.Button(tab_4, text="Run Automation", command=on_run_selenium, font=("Helvetica", 12), bg="#4CAF50", fg="white", relief="raised")
    run_button.pack(pady=10)


    # Finalizing GUI setup
    tab_control.pack(expand=1, fill="both")
    root.geometry("800x700")  # Adjust the window size as needed
    root.mainloop()

if __name__ == "__main__":
    create_gui()

